In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#import numpy as np # linear algebra
#import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!nvidia-smi

In [ ]:
!tar xvfz /kaggle/input/bert4keras-packages/packs.tgz

In [ ]:
!pip install bert4keras --no-deps --force --no-index --find-links=file:///kaggle/working/packages/
!pip install tensorboard_data_server tensorboard Keras_Preprocessing tf_estimator_nightly tensorboard_plugin_wit --no-deps --force --no-index --find-links=file:///kaggle/working/packages/
!pip install tensorflow keras --no-deps --force --no-index --find-links=file:///kaggle/working/packages/

## Initial env

In [ ]:
#!pip install bert4keras --no-deps --force
#!pip install tensorflow-gpu==2.8.0
#!pip install tensorflow==2.8.0

In [ ]:
!cp /opt/conda/lib/python3.10/site-packages/bert4keras/backend.py _b.py
!sed -i "18s/sys/#sys/" _b.py
!sed "18a\    from tensorflow import keras\n    import tensorflow.keras.backend as K" _b.py > _b2.py
!sed -i "22s/import/#import/" _b2.py
!sed -i "23s/import/#import/" _b2.py
!cp _b2.py /opt/conda/lib/python3.10/site-packages/bert4keras/backend.py

In [ ]:
#!curl -LJO https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
#!unzip uncased_L-12_H-768_A-12.zip -d bert_base

#!curl -LJO https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
#!unzip uncased_L-12_H-768_A-12.zip -d bert_large

## Convert raw data to train set

In [ ]:
import os
import random
import json
from tqdm import tqdm
from copy import deepcopy

train_file = '/kaggle/input/pii-detection-removal-from-educational-data/train.json'
test_file = '/kaggle/input/pii-detection-removal-from-educational-data/test.json'

split_ratio = 0.8


def __convert(indata, include_blank=True):

    text = ''
    entities = []
    all_idx = 0
    start_idx = 0
    etype = ''

    text = indata['sentence']

    for n, label in enumerate(indata['BIO_label']):
        if label[0]=='O':
            if etype!='':
                entities.append({
                    "start_idx": len(''.join(text[:start_idx])),
                    "end_idx": len((''.join(text[:all_idx])).rstrip()) - 1,
                    "type": etype,
                    "entity": (''.join(text[start_idx:all_idx])).rstrip(),
                })
            start_idx = 0
            etype = ''
        elif label[0]=='B':
            if etype!='':
                entities.append({
                    "start_idx": len(''.join(text[:start_idx])),
                    "end_idx": len((''.join(text[:all_idx])).rstrip()) - 1,
                    "type": etype,
                    "entity": (''.join(text[start_idx:all_idx])).rstrip(),
                })                
            start_idx = all_idx
            etype = label.split('-')[1]
        elif label[0]=='I':
            pass
        else:
            print('unknown label: ', label)

        all_idx += 1


    # 一行text结束
    if etype!='':
        entities.append({
            "start_idx": len(''.join(text[:start_idx])),
            "end_idx": len((''.join(text[:all_idx])).rstrip()) - 1,
            "type": etype,
            "entity": (''.join(text[start_idx:all_idx])).rstrip(),
        })

    # 加入数据集
    if include_blank or len(entities)>0:
        return {
            'text' : ''.join(text),
            'entities' : entities,
        }
    else:
        return None




def assemble(infile, outfile_path, max_len=500, is_train=True, include_blank=True):
    total = text_break = tmp_break = 0
    D = []

    data = json.load(open(infile))
    for l in tqdm(data):
        #print(f"---> {l['document']}")

        total += 1

        text = []
        tmp_text = []
        n = n_text = n_tmp = 0
        while n<len(l['tokens']):
            token = l['tokens'][n].replace('…', '.').replace('´', "'").replace('²', '2')\
                .replace('΅', "'").replace('¨', "'").replace(';', ';').replace('．', '.')\
                .replace('³', '3').replace('‑', '-').replace('¹', '1').replace('½', '1')\
                .replace('¾', '1').replace('¼', '1').replace('\xad', '-')\
                .replace('ﬄ', 'ffl').replace('ﬃ', 'ffi').replace('ﬂ', 'fl')\
                .replace('ﬁ', 'fi').replace('ﬀ', 'ff').replace('™', 'TM').replace('№', 'No')

            if l['trailing_whitespace'][n]:
                token += ' ' 

            if n_tmp + 1 > max_len:
                text += deepcopy(tmp_text)
                tmp_text = []
                n_text += n_tmp
                n_tmp = 0

                tmp_break += 1

                #print(text)
                #assert False, f"tmp_text is too long: {len(text)}, {len(tmp_text)}, {len(token)}"

            if n_text + n_tmp + 1 > max_len:
                assert n_text>0, f"too long: {n_text}, {n_tmp}, {max_len}"
                #print(text)
                #print('-'*20)
                dd = __convert({
                        'sentence'  : [x[0] for x in text],
                        'BIO_label' : [x[1] for x in text],
                    }, include_blank=include_blank)
                if dd:
                    dd['document'] = l['document']
                    if not is_train:
                        dd['tokens'] = [x[0] for x in text]
                    D.append(dd)
                text = []
                n_text = 0

                text_break += 1

            if is_train:
                tmp_text += [(token, l['labels'][n])] # token, label
            else:
                tmp_text += [(token, 'O')] # token, blank-label
            n_tmp += 1

            if token=='\n\n':
                text += deepcopy(tmp_text)
                tmp_text = []
                n_text += n_tmp
                n_tmp = 0

            n += 1

        if n_text + n_tmp > 0:
            text += deepcopy(tmp_text)
            n_text += n_tmp
            #print(text)
            #print('-'*20)
            dd = __convert({
                    'sentence'  : [x[0] for x in text],
                    'BIO_label' : [x[1] for x in text],
                }, include_blank=include_blank)
            if dd:
                dd['document'] = l['document']
                if not is_train:
                    dd['tokens'] = [x[0] for x in text]
                D.append(dd)

            text_break += 1            

        #break # for test

    blank = 0
    for x in D:
        if len(x['entities'])==0:
            blank += 1

    print(f"total= {total}\ttext_break= {text_break}\ttmp_break= {tmp_break}\tblank= {blank}")

    if is_train:
        random.shuffle(D)

        # 拆分数据集
        split_n = int(len(D) * split_ratio)

        json.dump(
            D[:split_n],
            open(os.path.join(outfile_path, "train.json"), 'w', encoding='utf-8'),
            indent=4,
            ensure_ascii=False
        )


        json.dump(
            D[split_n:],
            open(os.path.join(outfile_path, "dev.json"), 'w', encoding='utf-8'),
            indent=4,
            ensure_ascii=False
        )

        print(f"train set: {split_n}\tdev set: {len(D)-split_n}")

    else:
        json.dump(
            D,
            open(os.path.join(outfile_path, "test.json"), 'w', encoding='utf-8'),
            indent=4,
            ensure_ascii=False
        )

        print(f"test set: {len(D)}")




In [ ]:
assemble(train_file, '.', include_blank=False)
assemble(test_file, '.', is_train=False)

## Model

In [ ]:
#! -*- coding: utf-8 -*-
# 用GlobalPointer做中文命名实体识别

import os
os.environ["TF_KERAS"] = "1" # use tf 2.7 keras

import json
import math
import numpy as np
from bert4keras.backend import keras, K
from bert4keras.backend import multilabel_categorical_crossentropy
#from bert4keras.layers import GlobalPointer
from bert4keras.layers import EfficientGlobalPointer as GlobalPointer
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.optimizers import Adam, extend_with_exponential_moving_average
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open, to_array
from keras.models import Model
from keras.callbacks import LearningRateScheduler
from tqdm import tqdm

maxlen = 512
epochs = 10
batch_size = 4
learning_rate = 1e-5
categories = set()

# bert配置
config_path = '/kaggle/input/google_bert/tensorflow2/bert_large_uncased_pii_43k_noblank/1/bert_config.json'
checkpoint_path = None #'/kaggle/input/google_bert/tensorflow2/bert_large_uncased_pii_43k_noblank/1/bert_model.ckpt'
dict_path = '/kaggle/input/google_bert/tensorflow2/bert_large_uncased_pii_43k_noblank/1/vocab.txt'

def load_data(filename):
    """加载数据
    单条格式：[text, (start, end, label), (start, end, label), ...]，
              意味着text[start:end + 1]是类型为label的实体。
    """
    D = []
    for d in json.load(open(filename)):
        D.append([d['text']])
        for e in d['entities']:
            start, end, label = e['start_idx'], e['end_idx'], e['type']
            if start <= end:
                D[-1].append((start, end, label))
            categories.add(label)
    return D


# 标注数据
train_data = load_data('./train.json')
valid_data = load_data('./dev.json')
categories = list(sorted(categories))

print("labels: ", categories)
# labels:  ['EMAIL', 'ID_NUM', 'NAME_STUDENT', 'PHONE_NUM', 'STREET_ADDRESS', 'URL_PERSONAL', 'USERNAME']


# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)


class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, d in self.sample(random):
            tokens = tokenizer.tokenize(d[0], maxlen=maxlen)
            mapping = tokenizer.rematch(d[0], tokens)
            start_mapping = {j[0]: i for i, j in enumerate(mapping) if j}
            end_mapping = {j[-1]: i for i, j in enumerate(mapping) if j}
            token_ids = tokenizer.tokens_to_ids(tokens)
            segment_ids = [0] * len(token_ids)
            labels = np.zeros((len(categories), maxlen, maxlen))
            for start, end, label in d[1:]:
                if start in start_mapping and end in end_mapping:
                    start = start_mapping[start]
                    end = end_mapping[end]
                    label = categories.index(label)
                    labels[label, start, end] = 1
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append(labels[:, :len(token_ids), :len(token_ids)])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels, seq_dims=3)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []


def global_pointer_crossentropy(y_true, y_pred):
    """给GlobalPointer设计的交叉熵
    """
    bh = K.prod(K.shape(y_pred)[:2])
    y_true = K.reshape(y_true, (bh, -1))
    y_pred = K.reshape(y_pred, (bh, -1))
    return K.mean(multilabel_categorical_crossentropy(y_true, y_pred))


def global_pointer_f1_score(y_true, y_pred, epsilon=1e-10):
    """给GlobalPointer设计的F1
    """
    y_pred = K.cast(K.greater(y_pred, 0), K.floatx())
    return 2 * K.sum(y_true * y_pred) / (K.sum(y_true + y_pred) + epsilon)


model = build_transformer_model(config_path, checkpoint_path)
output = GlobalPointer(len(categories), 64)(model.output)

AdamEMA = extend_with_exponential_moving_average(Adam, name='AdamEMA')
optimizer = AdamEMA(learning_rate=learning_rate)

model = Model(model.input, output)
model.summary()

model.compile(
    loss=global_pointer_crossentropy,
    optimizer=optimizer, #Adam(learning_rate),
    #optimizer=Adam(learning_rate),
    metrics=[global_pointer_f1_score]
)


class NamedEntityRecognizer(object):
    """命名实体识别器
    """
    def recognize(self, text, threshold=0):
        tokens = tokenizer.tokenize(text, maxlen=512)
        mapping = tokenizer.rematch(text, tokens)
        token_ids = tokenizer.tokens_to_ids(tokens)
        segment_ids = [0] * len(token_ids)
        token_ids, segment_ids = to_array([token_ids], [segment_ids])
        scores = model.predict([token_ids, segment_ids])[0]
        scores[:, [0, -1]] -= np.inf
        scores[:, :, [0, -1]] -= np.inf
        entities = []
        for l, start, end in zip(*np.where(scores > threshold)):
            entities.append(
                (mapping[start][0], mapping[end][-1], categories[l])
            )
        return entities


NER = NamedEntityRecognizer()


def evaluate(data):
    """评测函数
    """
    X, Y, Z = 1e-10, 1e-10, 1e-10
    for d in tqdm(data, ncols=100):
        R = set(NER.recognize(d[0]))
        T = set([tuple(i) for i in d[1:]])
        X += len(R & T)
        Y += len(R)
        Z += len(T)
    f1, precision, recall = 2 * X / (Y + Z), X / Y, X / Z
    return f1, precision, recall


class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_f1 = 0

    def on_epoch_end(self, epoch, logs=None):
        f1, precision, recall = evaluate(valid_data)
        # 保存最优
        if f1 >= self.best_val_f1:
            self.best_val_f1 = f1
            model.save_weights('./pii_gp_best_f1.h5')
        print(
            'valid:  f1: %.5f, precision: %.5f, recall: %.5f, best f1: %.5f\n' %
            (f1, precision, recall, self.best_val_f1)
        )


def predict_to_file(in_file, out_file):
    """预测到文件
    """
    data = json.load(open(in_file))

    document = ""
    last_pos = 0
    D = []

    for d in tqdm(data, ncols=100):

        if document != d['document']:
            document = d['document']
            last_pos = 0

        # 初始化 BIO 标记
        label = ['O']*len(d['tokens'])

        # 识别
        entities = NER.recognize(d['text'])
        for e in entities:
            d['entities'].append({
                'start_idx': e[0],
                'end_idx': e[1],
                'type': e[2]
            })

            # 生成 BIO标记, 依据 原始 tokens
            pos = last = 0
            for n, x in enumerate(d['tokens']):
                if pos >= e[0] and pos <= e[1]+1:
                    if last==0:
                        label[n] = 'B-'+e[2]
                        last += 1
                    else:
                        label[n] = 'I-'+e[2]
                    D.append((document, last_pos+n, label[n]))
                else:
                    last = 0

                pos += len(x)

                if pos > e[1]:
                    break

        d['labels'] = label

        last_pos += len(d['tokens'])

    # 保存json格式
    json.dump(
        data,
        open("output.json", 'w', encoding='utf-8'),
        indent=4,
        ensure_ascii=False
    )

    with open(out_file, "w") as f:
        f.write("row_id,document,token,label\n")
        for n, x in enumerate(D):
            f.write(f"{n},{x[0]},{x[1]},{x[2]}\n")

def lr_step_decay(epoch):
    drop = 0.8
    epochs_drop = 1.0
    lrate = learning_rate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

## Train

In [ ]:
# 训练
batch_size = 2
epochs = 10

evaluator = Evaluator()
lrate = LearningRateScheduler(lr_step_decay)
train_generator = data_generator(train_data, batch_size)

model.load_weights('/kaggle/input/google_bert/tensorflow2/bert_large_uncased_pii_43k_noblank/1/pii_gp_best_f1_0.92609.h5')

model.fit(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    callbacks=[evaluator, lrate]
)

## Generate result

In [ ]:
# 推理生成结果

model.load_weights('pii_gp_best_f1.h5')
predict_to_file('test.json', 'submission.csv')